## Import Libraries

In [16]:
# %matplotlib ipympl
# %matplotlib inline
%matplotlib wx

In [17]:
import matplotlib.pyplot as plt
plt.ion()

In [18]:
from pydgilib_extra import *
from atprogram.atprogram import atprogram

In [19]:
from os import path, pardir

## Compile and program project

In [20]:
project_path = [path.curdir, "Dijkstra-S"]
project_path

['.', 'Dijkstra-S']

In [21]:
atprogram(path.abspath(path.join(*project_path)), verbose=2)

make: Nothing to be done for 'all'.

[DEBUG] Starting execution of "chiperase"
[DEBUG] Starting process 'C:\Program Files (x86)\Atmel\Studio\7.0\atbackend\atbackend.exe'
[DEBUG] Connecting to TCP:127.0.0.1:8338
[INFO] Connected to edbg, fw version: 3.25
[INFO] Firmware check OK
[DEBUG] Command "chiperase" finished with return code 0
[DEBUG] Starting execution of "program"
[DEBUG] Memory segment base written at 0x00000000. Size = 0x00000d90.
[DEBUG] Memory segment base written at 0x00000d90. Size = 0x00000064.
[DEBUG] Memory segment base written at 0x0000fb00. Size = 0x00000020.
[DEBUG] Memory segment base written at 0x00804000. Size = 0x00000020.
[DEBUG] Command "program" finished with return code 0
[DEBUG] Exit successfully.
Firmware check OK
Chiperase completed successfully
Programming completed successfully.



0

## Data Logging

In [22]:
live_plot = True

Create a figure for the plot.

In [23]:
if live_plot:
    fig = plt.figure(figsize=(10, 6))
    fig.show()

Create the configuration dictionary for `DGILibExtra`.

In [24]:
config_dict = {
    "loggers": [LOGGER_OBJECT, LOGGER_CSV],
    "file_name_base": "experiment_dijkstra"
}
config_dict_plot = {
    "loggers": [LOGGER_OBJECT, LOGGER_PLOT, LOGGER_CSV],
    "plot_pins": [True, True, True, True],
    "plot_pins_method": "line",
    "plot_xmax": 1,
    "window_title": "Experiment Dijkstra",
}

Stop criteria to pass to the logger:

In [25]:
def stop_fn(logger_data):
    return all(logger_data.gpio.values[-1])

Stop criteria to pass to the parser:

In [26]:
def stop_function(pin_values):
    return all(pin_values)

In [27]:
repetitions = 10

Perform the measurement.

In [28]:
data = []
cd = config_dict.copy()
if live_plot:
    fig.clf()
    for ax in fig.get_axes():
        ax.cla()
    
    cd.update(config_dict_plot)
    cd["fig"] = fig
        
charges = []
times = []

with DGILibExtra(**cd) as dgilib:
    for _ in range(repetitions):
        dgilib.device_reset()
        dgilib.logger.log(2, stop_fn)
        print(dgilib.data)
        charge, time = power_and_time_per_pulse(dgilib.data, 2, stop_function=stop_function)
        charges += charge
        times += time
        if len(charge) > 1 or len(time) > 1:
            print(f"Parsing of measurement data failed, charge:{charge}, time:{time}")
            break
        dgilib.logger.plotobj.ax.set_title(f"Previous sample: charge: {charge[0]*1e3:.06} mC, time: {time[0]:.06} s")
        dgilib.empty_data()
    
    dgilib.logger.plotobj.ax.set_title(f"Average of {repetitions} samples: charge: {sum(charges)/len(charges)*1e3:.06} mC, time: {sum(times)/len(times):.06} s")

Interfaces:
	  48:   gpio,    samples:       2
	 256:   power,   samples:    7000



IndexError: list index out of range

In [ ]:
import json

config = {}
config["name"] = "Dijkstra 64"
config["project_paths"] = [project_path]
config["config_dict"] = config_dict
config["config_dict_plot"] = config_dict_plot
config["analysis"] = {"pins":{2: ["Dijkstra"]}, 
                      "result_types": ["Charge", "Time"], 
                      "section_types": {"init": [], 
                                        "loop": ["Dijkstra"],
                                        "exit": []}}

with open("repeated_experiment.json", 'w') as config_file:  
    json.dump(config, config_file, indent=4)

## Generate test data

In [8]:
import numpy as np

In [9]:
from collections import namedtuple, deque
 
 
inf = float('inf')
Edge = namedtuple('Edge', 'start, end, cost')
 
class Graph():
    def __init__(self, edges):
        self.edges = edges2 = [Edge(*edge) for edge in edges]
        self.vertices = set(sum(([e.start, e.end] for e in edges2), []))
 
    def dijkstra(self, source, dest):
        assert source in self.vertices
        dist = {vertex: inf for vertex in self.vertices}
        previous = {vertex: None for vertex in self.vertices}
        dist[source] = 0
        q = self.vertices.copy()
        neighbours = {vertex: set() for vertex in self.vertices}
        for start, end, cost in self.edges:
            neighbours[start].add((end, cost))
        #pp(neighbours)
 
        while q:
            u = min(q, key=lambda vertex: dist[vertex])
            q.remove(u)
            if dist[u] == inf or u == dest:
                break
            for v, cost in neighbours[u]:
                alt = dist[u] + cost
                if alt < dist[v]: # Relax (u,v,a)
                    dist[v] = alt
                    previous[v] = u
        #pp(previous)
        s, u = deque(), dest
        while previous[u]:
            s.appendleft(u)
            u = previous[u]
        s.appendleft(u)
        return s

In [13]:
class CGraph(Graph):
    def print_c(self):
        print('int main (void) {\n\tgraph_t *g = calloc(1, sizeof (graph_t));')
        for edge in self.edges:
            print(f'\tadd_edge(g, {edge.start}, {edge.end}, {edge.cost});')
        end_node = len(self.vertices) - 1
        print('\tdijkstra(g, 0, ' + f'{end_node}' + ');\n#ifdef DEBUG_PRINT\n\tprint_path(g, ' + f'{end_node}' + ');\n#endif // DEBUG_PRINT\n\treturn 0;\n}')
        
    def print_path(self):
        path = self.dijkstra(0, len(self.vertices) - 1)
        s = f'hops: {len(path)}, path: '
        for e in reversed(path):
            s += f'{e}<-'
        print(s + '0')

def random_edges(nodes, nedges):
    idx = np.random.choice(np.prod((nodes,nodes-1)), nedges, replace=False)
    diags = np.array([(i + i * (nodes - 1)) for i in range(nodes)])
    return np.vstack(np.unravel_index(idx + np.searchsorted(diags, idx+1), (nodes,nodes))).T

def random_graph(nodes, nedges):
    edges = random_edges(nodes, nedges)
    graph = CGraph([(*edge, np.random.randint(int_max)) for edge in edges])
    return graph

In [14]:
np.random.seed(314)
nodes = 40
nedges = 90
int_max = 255
graph = random_graph(nodes, nedges)

In [15]:
graph.print_path()
graph.print_c()

hops: 6, path: 39<-10<-35<-32<-37<-34<-0
int main (void) {
	graph_t *g = calloc(1, sizeof (graph_t));
	add_edge(g, 36, 21, 147);
	add_edge(g, 38, 21, 176);
	add_edge(g, 26, 22, 223);
	add_edge(g, 9, 27, 163);
	add_edge(g, 10, 39, 249);
	add_edge(g, 11, 33, 53);
	add_edge(g, 39, 5, 98);
	add_edge(g, 20, 12, 171);
	add_edge(g, 16, 38, 214);
	add_edge(g, 0, 13, 206);
	add_edge(g, 21, 9, 105);
	add_edge(g, 0, 34, 178);
	add_edge(g, 14, 0, 92);
	add_edge(g, 12, 23, 80);
	add_edge(g, 1, 6, 191);
	add_edge(g, 34, 7, 107);
	add_edge(g, 38, 25, 188);
	add_edge(g, 6, 31, 129);
	add_edge(g, 22, 13, 181);
	add_edge(g, 7, 24, 21);
	add_edge(g, 18, 31, 254);
	add_edge(g, 15, 6, 30);
	add_edge(g, 19, 23, 159);
	add_edge(g, 16, 7, 141);
	add_edge(g, 34, 25, 198);
	add_edge(g, 38, 29, 101);
	add_edge(g, 16, 11, 41);
	add_edge(g, 10, 14, 3);
	add_edge(g, 21, 11, 67);
	add_edge(g, 1, 30, 137);
	add_edge(g, 4, 9, 179);
	add_edge(g, 1, 8, 163);
	add_edge(g, 34, 6, 122);
	add_edge(g, 19, 15, 86);
	add_edge(

## Analysis

In [14]:
import pandas as pd

In [15]:
df = pd.DataFrame({'times': times, 'charges':charges})

In [16]:
df

,times,charges
0,0.022706,0.000007
1,0.022639,0.000007
2,0.022640,0.000007
3,0.022705,0.000007
4,0.022639,0.000007
5,0.022639,0.000007
6,0.022639,0.000007
7,0.022637,0.000007
8,0.022703,0.000007
9,0.022703,0.000007


In [17]:
df.describe()

,times,charges
count,10.000000,1.000000e+01
mean,0.022665,6.619365e-06
std,0.000034,1.219877e-08
min,0.022637,6.592197e-06
25%,0.022639,6.614269e-06
50%,0.022640,6.620769e-06
75%,0.022703,6.627945e-06
max,0.022706,6.634523e-06


In [18]:
sum(charges)/repetitions

6.619364574372876e-06

In [19]:
print("%.0100f" % (sum(charges)/repetitions))

0.0000066193645743728758915470737622666064225995796732604503631591796875000000000000000000000000000000


4096

In [40]:
# https://rosettacode.org/wiki/Dijkstra%27s_algorithm#Python

from collections import namedtuple, deque
 
 
inf = float('inf')
Edge = namedtuple('Edge', 'start, end, cost')
 
class Graph():
    def __init__(self, edges):
        self.edges = edges2 = [Edge(*edge) for edge in edges]
        self.vertices = set(sum(([e.start, e.end] for e in edges2), []))
 
    def dijkstra(self, source, dest):
        assert source in self.vertices
        dist = {vertex: inf for vertex in self.vertices}
        previous = {vertex: None for vertex in self.vertices}
        dist[source] = 0
        q = self.vertices.copy()
        neighbours = {vertex: set() for vertex in self.vertices}
        for start, end, cost in self.edges:
            neighbours[start].add((end, cost))
        # print(neighbours)
 
        while q:
            u = min(q, key=lambda vertex: dist[vertex])
            q.remove(u)
            if dist[u] == inf or u == dest:
                break
            for v, cost in neighbours[u]:
                alt = dist[u] + cost
                if alt < dist[v]:  # Relax (u,v,a)
                    dist[v] = alt
                    previous[v] = u
        # print(previous)
        s, u = deque(), dest
        while previous[u]:
            s.appendleft(u)
            u = previous[u]
        s.appendleft(u)
        return s

In [41]:
l = []
for i in range(NUM_NODES):
    for j in range(NUM_NODES):
        if i != j:
            l.append((i,j,randints[i*NUM_NODES+j] if randints[i*NUM_NODES+j] else float('inf')))
l

[(0, 1, inf),
 (0, 2, inf),
 (0, 3, inf),
 (0, 4, inf),
 (0, 5, inf),
 (0, 6, inf),
 (0, 7, inf),
 (0, 8, inf),
 (0, 9, inf),
 (0, 10, inf),
 (0, 11, inf),
 (0, 12, inf),
 (0, 13, inf),
 (0, 14, inf),
 (0, 15, inf),
 (0, 16, inf),
 (0, 17, inf),
 (0, 18, inf),
 (0, 19, inf),
 (0, 20, inf),
 (0, 21, inf),
 (0, 22, inf),
 (0, 23, inf),
 (0, 24, inf),
 (0, 25, inf),
 (0, 26, inf),
 (0, 27, inf),
 (0, 28, 111),
 (0, 29, inf),
 (0, 30, inf),
 (0, 31, 54),
 (0, 32, inf),
 (0, 33, inf),
 (0, 34, inf),
 (0, 35, inf),
 (0, 36, inf),
 (0, 37, inf),
 (0, 38, inf),
 (0, 39, inf),
 (0, 40, inf),
 (0, 41, inf),
 (0, 42, inf),
 (0, 43, inf),
 (0, 44, inf),
 (0, 45, inf),
 (0, 46, inf),
 (0, 47, inf),
 (0, 48, inf),
 (0, 49, inf),
 (0, 50, inf),
 (0, 51, inf),
 (0, 52, inf),
 (0, 53, inf),
 (0, 54, inf),
 (0, 55, inf),
 (0, 56, inf),
 (0, 57, inf),
 (0, 58, inf),
 (0, 59, inf),
 (0, 60, inf),
 (0, 61, inf),
 (0, 62, inf),
 (0, 63, inf),
 (0, 64, inf),
 (0, 65, inf),
 (0, 66, inf),
 (0, 67, 124),
 (0, 

In [42]:
sp = Graph(l).dijkstra(0,NUM_NODES-1)
sp_str = 'path='
for n in reversed(list(sp)):
    sp_str += f'{n}<-'
sp_str += '0'
sp_str

'path=255<-200<-84<-240<-0'